In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd 
import nltk
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sns
from nltk.corpus import stopwords
import plotly.graph_objs as go
from plotly import tools
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import random
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.pipeline import make_pipeline
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
def random_colors(number_of_colors):
    color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
                 for i in range(number_of_colors)]
    return color
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import re, os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv('/kaggle/input/twitter-hate-speech/train_E6oV3lV.csv')

In [ ]:
train.head()

In [ ]:
train.groupby('label').describe()

In [ ]:
train['length'] = train['tweet'].apply(len)
train.head()

In [ ]:
count_Class = pd.value_counts(train.label, sort = True)

# Data to Plot
labels = '0', '1'
sizes = [count_Class[0], count_Class[1]]
colors = ['red', 'skyblue']
explode = (0.1, 0.1)

# Plot
plt.pie(sizes, explode = explode, labels = labels, colors = colors,
        autopct = '%1.1f%%', shadow = True, startangle = 90)
plt.axis('equal')
plt.show()

In [ ]:
species_count = train['label'].value_counts()
data = [go.Bar(
    x = species_count.index,
    y = species_count.values,
    marker = dict(color = random_colors(3))
)]
py.iplot(data)

In [ ]:
train['cleaned_tweet'] = train.tweet.apply(lambda x: ' '.join([word for word in x.split() if not word.startswith('@')]))
train['hashtags'] = train.tweet.apply(lambda x: ' '.join([word[1:] for word in x.split() if word.startswith('#')]))


In [ ]:
train.head()

In [ ]:
#Select all words from normal tweet
normal_words = ' '.join([word for word in train['cleaned_tweet']])
#Collect all hashtags
pos_htag = [htag for htag in normal_words.split() if htag.startswith('#')]
#Remove hashtag symbol (#)
pos_htag = [pos_htag[i][1:] for i in range(len(pos_htag))]
#Count frequency of each word
pos_htag_freqcount = nltk.FreqDist(pos_htag)
pos_htag_df = pd.DataFrame({'Hashtag' : list(pos_htag_freqcount.keys()),
                            'Count' : list(pos_htag_freqcount.values())})

In [ ]:
#Select top 20 most frequent hashtags and plot them   
most_frequent = pos_htag_df.nlargest(columns="Count", n = 20) 
plt.figure(figsize=(16,5))
ax = sns.barplot(data=most_frequent, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

In [ ]:
def preprocess_text(sen):
    sentence = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)',' ',sen) # Removing html tags
    sentence = re.sub('[^a-zA-Z]', ' ', sentence) # Remove punctuations and numbers
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence) # Single character removal
    sentence = re.sub(r'\s+', ' ', sentence) # Removing multiple spaces
    sentence = sentence.replace("ain't", "am not").replace("aren't", "are not")
    sentence = ' '.join(text.lower() for text in sentence.split(' ')) # Lowering cases
    sw = stopwords.words('english')
    sentence = ' '.join(text for text in sentence.split() if text not in sw) #removing stopwords
    #sentence = ' '.join(text.lemmatize() for text in sentence.split()) #lemmatization
    return sentence

In [ ]:
train['cleaned_tweet'] = train.tweet.apply(preprocess_text)

In [ ]:
train['cleaned_tweet'] = train['cleaned_tweet'].apply(nltk.word_tokenize)

In [ ]:
stemmer = PorterStemmer()
train['cleaned_tweet'] = train['cleaned_tweet'].apply(lambda x: [stemmer.stem(y) for y in x])

In [ ]:
train['cleaned_tweet']

In [ ]:
train['cleaned_tweet'] = train['cleaned_tweet'].apply(lambda x: ' '.join(x))
count_vect = CountVectorizer()
counts = count_vect.fit_transform(train['cleaned_tweet']);

In [ ]:
transformer = TfidfTransformer().fit(counts)
counts = transformer.fit_transform(counts)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(counts, train['label'], test_size=0.3, random_state=69)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# implementing naive bayes
NB = MultinomialNB().fit(X_train, y_train)

In [ ]:
predicted = NB.predict(X_test)

print(np.mean(predicted == y_test))

In [ ]:
print(confusion_matrix(y_test, predicted))

In [ ]:
print(classification_report(y_test,predicted))

In [ ]:
# implementing disession tree

dt = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
predicted = dt.predict(X_test)

print(np.mean(predicted == y_test))

In [ ]:
print(classification_report(y_test,predicted))

In [ ]:
print(confusion_matrix(y_test, predicted))

In [ ]:
# implementing adabost 
ab = AdaBoostClassifier().fit(X_train, y_train)

In [ ]:
predicted = ab.predict(X_test)

print(np.mean(predicted == y_test))

In [ ]:
print(classification_report(y_test,predicted))

In [ ]:
print(confusion_matrix(y_test, predicted))